In [0]:
SET statement_timeout = 0;
/*------------------------------------------------------------
  1.  Schema
------------------------------------------------------------*/
CREATE SCHEMA IF NOT EXISTS bench;
SET search_path TO bench;

/*------------------------------------------------------------
  2.  Table
------------------------------------------------------------*/
CREATE TABLE  IF NOT EXISTS transactions (
    txn_id       BIGSERIAL PRIMARY KEY,          -- 64‑bit monotonically‑increasing id
    account_id   INTEGER      NOT NULL,          -- who made / received the txn
    amount_cents INTEGER      NOT NULL,          -- amount in smallest currency unit
    currency     CHAR(3)      NOT NULL,          -- ISO‑4217, e.g. 'USD'
    txn_ts       TIMESTAMPTZ  NOT NULL DEFAULT now(),
    note         TEXT
);

/*------------------------------------------------------------
  3.  Bulk‑insert 1 000 000 rows
      – pure SQL using generate_series()
      – about 1–2 seconds on a typical laptop SSD
------------------------------------------------------------*/
/* -----------------------------------------------------------------
   Robust 1 000 000‑row insert — currency is *never* NULL now
   ----------------------------------------------------------------- */
INSERT INTO bench.transactions
        (account_id, amount_cents, currency,   txn_ts,                         note)
SELECT  (floor(random()*100000) + 1)::int      AS account_id,          -- 1‑100 000
        (floor(random()*199999)  - 99999)::int AS amount_cents,        -- −99 999…+99 999
        (ARRAY['USD','EUR','GBP'])
        [ (floor(random()*3) + 1)::int ]       AS currency,            -- always 1‑3
        now() - (interval '1 day' * floor(random()*730)) AS txn_ts,    -- past 730 days
        'synthetic row #' || gs                AS note
FROM generate_series(1, 1000000) AS gs;
-- About 1M rows inserted

/*------------------------------------------------------------
  4.  Additional indexes
------------------------------------------------------------*/
-- Most workloads fetch by account_id; make it a b‑tree index
CREATE INDEX IF NOT EXISTS idx_transactions_account_id
    ON transactions (account_id);

-- Example composite index if you often query "last N txns for one account"
CREATE INDEX IF NOT EXISTS idx_transactions_account_ts
    ON transactions (account_id, txn_ts DESC);

/*------------------------------------------------------------
  5.  Quick sanity checks & timing examples
------------------------------------------------------------*/

-- Expect exactly 1 000 000 rows
SELECT COUNT(*) FROM transactions;

-- Disable seqscan so you can see the index being used
SET enable_seqscan = off;

-- Primary‑key search (fast, uses transactions_pkey)
EXPLAIN ANALYZE
SELECT * FROM bench.transactions WHERE txn_id = 40000;

-- Account‑id search (fast, uses idx_transactions_account_id)
EXPLAIN ANALYZE
SELECT * FROM bench.transactions WHERE account_id = 12345 LIMIT 1;

-- Restore planner defaults
RESET enable_seqscan;
